In [8]:
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from modules.lifter_2d_3d.dataset.drive_and_act_keypoint_dataset import DriveAndActKeypointDataset
from modules.lifter_2d_3d.model.linear_model.lit_linear_model import LitSimpleBaselineLinear
from modules.utils.visualization import (
    plot_samples
)
from IPython.display import display
from pathlib import Path


pl.seed_everything(1234)

# ------------
# dataset path
# ------------
dataset_root_path = Path('/root/data/processed/drive_and_act/')
keypoint_2d_path = dataset_root_path / 'keypoint_detection_results'
keypoint_3d_path = dataset_root_path / 'annotations'
bbox_path = dataset_root_path / 'person_detection_results'
# ------------
# model
# ------------
image_width = 1280
image_height = 1024
batch_size = 64
max_epoch = 200
val_check_period = 5
early_stopping_patience = 5
# ------------
# saved model path
# ------------
# pretrained_model_path = './saved_lifter_2d_3d_model/linear_model/synthetic_cabin_ir_1m/A_Pillar_Codriver/gt_bbox'
# pretrained_model_path = './saved_lifter_2d_3d_model/linear_model/synthetic_cabin_ir_1m/A_Pillar_Codriver/gt_2d'
with open(f'{pretrained_model_path}/best_model_path.txt', 'r') as f:
    checkpoint_path = f.readline()


Global seed set to 1234


In [9]:
def run_experiment(
    train_actors,
    val_actors,
    test_actors,
    is_plot_gt_skeleton=False,
    model_suffix=''
):
    train_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        bbox_file=(bbox_path / 'human_detection_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=train_actors,
        exclude_ankle=True,
        exclude_knee=True,
        is_center_to_neck=True,
        is_normalize_to_bbox=False,
        is_normalize_to_pose=True,
        is_normalize_rotation=True
    )
    val_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        bbox_file=(bbox_path / 'human_detection_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=val_actors,
        exclude_ankle=True,
        exclude_knee=True,
        is_center_to_neck=True,
        is_normalize_to_bbox=False,
        is_normalize_to_pose=True,
        is_normalize_rotation=True
    )
    test_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        bbox_file=(bbox_path / 'human_detection_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=test_actors,
        exclude_ankle=True,
        exclude_knee=True,
        is_center_to_neck=True,
        is_normalize_to_bbox=False,
        is_normalize_to_pose=True,
        is_normalize_rotation=True
    )

    print(
        'train_dataset', len(train_dataset),
        'val_dataset', len(val_dataset),
        'test_dataset', len(test_dataset)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=24)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True, num_workers=24)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=24)

    all_activities = train_dataset.activities.union(val_dataset.activities).union(test_dataset.activities)
    # lit_model = LitSimpleBaselineLinear(exclude_ankle=True, exclude_knee=True, all_activities=all_activities)
    lit_model = LitSimpleBaselineLinear.load_from_checkpoint(checkpoint_path)
    lit_model.all_activities = all_activities
    model_checkpoint = ModelCheckpoint(monitor='val_loss',mode='min', save_top_k=1)
    early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=early_stopping_patience)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    saved_model_path = f'./saved_lifter_2d_3d_model/linear_model/drive_and_act/transfer_learning/synthetic_cabin_ir_1m/A_Pillar_Codriver/gt_2d/all_actors/linear_model_{model_suffix}'
    if not os.path.exists(saved_model_path):
        os.makedirs(saved_model_path)
    trainer = pl.Trainer(
        # max_steps=10,
        max_epochs=max_epoch,
        callbacks=[model_checkpoint, early_stopping],
        accelerator=device,
        check_val_every_n_epoch=val_check_period,
        default_root_dir=saved_model_path,
        gradient_clip_val=1.0
    )
    trainer.fit(lit_model, train_loader, val_loader)
    with open(f'{saved_model_path}/best_model_path.txt', 'w') as f:
        f.writelines(model_checkpoint.best_model_path)
    best_checkpoint_path = model_checkpoint.best_model_path
    trainer.test(ckpt_path=best_checkpoint_path, dataloaders=test_loader)
    return trainer

## Fine-Tuned with full dataset

In [10]:
trainer = run_experiment(
    train_actors=['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8'],
    val_actors=['vp9', 'vp10', 'vp15'],
    test_actors=['vp11', 'vp12', 'vp13', 'vp14'],
    is_plot_gt_skeleton=False,
    model_suffix='all_actor'
)

skipping problematic image 3628
skipping problematic image 5874
skipping problematic image 14835
skipping problematic image 15783
skipping problematic image 17258
skipping problematic image 17259
skipping problematic image 21271
skipping problematic image 21272
skipping problematic image 21273
skipping problematic image 21274
skipping problematic image 21275
skipping problematic image 21276
skipping problematic image 32959
skipping problematic image 33527
skipping problematic image 28113


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: saved_lifter_2d_3d_model/linear_model/drive_and_act/transfer_learning/synthetic_cabin_ir_1m/A_Pillar_Codriver/gt_2d/all_actors/linear_model_all_actor/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | BaselineModel | 4.3 M 
----------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.105    Total estimated model params size (MB)


train_dataset 22881 val_dataset 6240 test_dataset 11017


Sanity Checking: 0it [00:00, ?it/s]

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/opt/conda/lib/python3.10/shutil.py", line 731, in rmtree
    onerror(os.rmdir, path, sys.exc_info())
  File "/opt/conda/lib/python3.10/shutil.py", line 729, in rmtree
    os.rmdir(path)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-__2zrj9o'


...

check #19
training loss from 1785 batches: 99.08654361921532
val MPJPE from: 6208 samples : 73.30895960330963
activity_macro_mpjpe nan


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at saved_lifter_2d_3d_model/linear_model/drive_and_act/transfer_learning/synthetic_cabin_ir_1m/A_Pillar_Codriver/gt_2d/all_actors/linear_model_all_actor/lightning_logs/version_0/checkpoints/epoch=69-step=24990.ckpt


Testing: 0it [00:00, ?it/s]

MPJPE: 63.21757002579432
PJPE
                      PJPE
nose             55.361515
left_eye         52.606520
right_eye        50.853244
left_ear         14.049860
right_ear        36.105998
left_shoulder    17.977145
right_shoulder   21.257364
left_elbow       66.452805
right_elbow      65.865112
left_wrist       82.850952
right_wrist     101.834413
left_hip         64.275006
right_hip        60.928563
activities_mpjpe:
{}
test mpjpe: 63.21757002579432
activity_macro_mpjpe nan


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           mpjpe           │     63.21757002579432     │
└───────────────────────────┴───────────────────────────┘

In [11]:
!ls /workspace/saved_lifter_2d_3d_model/linear_model/synthetic_cabin_ir_1m/A_Pillar_Codrive

ls: cannot access '/workspace/saved_lifter_2d_3d_model/linear_model/synthetic_cabin_ir_1m/A_Pillar_Codrive': No such file or directory


In [12]:
trainer.model.test_history[0]['mpjpe']

63.21757002579432

In [13]:
trainer.model.test_history[0]['pjpe']

,PJPE
nose,55.361515
left_eye,52.606520
right_eye,50.853244
left_ear,14.049860
right_ear,36.105998
left_shoulder,17.977145
right_shoulder,21.257364
left_elbow,66.452805
right_elbow,65.865112
left_wrist,82.850952


In [14]:
pd.DataFrame(trainer.model.test_history[0]['activities_mpjpe'], index=['mpjpe']).T.sort_index()

,mpjpe
